In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt

In [ ]:
imu1 = pd.read_csv('../input/cleaned-imu/imu_train_0_8.csv')
imu1

In [ ]:
acce = imu1.loc[imu1['Type'] == 'TYPE_ACCELEROMETER']
gyro = imu1.loc[imu1['Type'] == 'TYPE_GYROSCOPE']
magn = imu1.loc[imu1['Type'] == 'TYPE_MAGNETIC_FIELD']
rota = imu1.loc[imu1['Type'] == 'TYPE_ROTATION_VECTOR']

In [ ]:
magn

In [ ]:
acce

Post-processing based on waypoints

In [ ]:
train_waypoints = pd.read_csv('../input/waypoint/waypoint_train (1).csv')
# sub = sub_process(pd.read_csv('../input/indoor-location-train-waypoints/6.578LB_submission.csv'),
#                  train_waypoints)


In [ ]:
train_waypoints
waypoint_df = train_waypoints[['Time','x','y']]
waypoint_df

In [ ]:
!git clone https://github.com/location-competition/indoor-location-competition-20.git api

from api.compute_f import *
from api.io_f import * 
from api.visualize_f import * 

import json
from pathlib import Path

In [ ]:
# I copied these visualization functions mainly from https://www.kaggle.com/hrshtt/kalman-filters-and-imus-starter and modified them slightly. 

def visualize_trajectory(trajectory, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False, trajectory2=None):
    """
    Copied and modified from from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py

    """
    fig = go.Figure()

    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ['rgba(4, 174, 4, 0.5)'] * trajectory.shape[0]
    color_list[0] = 'rgba(12, 5, 235, 1)'
    color_list[-1] = 'rgba(235, 5, 5, 1)'
    
    

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append('        ' * position_count[str(trajectory[i])] + f'{i}')
    text_list[0] = 'Start 0'
    text_list[-1] = f'End {trajectory.shape[0] - 1}'

    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape='linear', color='lightgrey', width=3, dash='dash'),
            text=text_list,
            textposition="top center",
            name='trajectory',
        ))
    
    if trajectory2 is not None:
        size_list2 = [6] * trajectory2.shape[0]
        size_list2[0] = 10
        size_list2[-1] = 10
        
        color_list2 = ['rgba(4, 4, 174, 0.5)'] * trajectory2[0].shape[0]
        color_list2[0] = 'rgba(235, 5, 5, 1)'
        color_list2[-1] = 'rgba(5, 235, 5, 1)'
        fig.add_trace(
            go.Scattergl(
                x=trajectory2[:, 0],
                y=trajectory2[:, 1],
                mode=mode,
                marker=dict(size=size_list2, color=color_list2),
                line=dict(shape='linear', color='red', width=3, dash='dash'),
                #text=text_list,
                textposition="top center",
                name='trajectory',
            ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=  800 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig

def visualize_train_trajectory(txt_path):
    """
    Edited from 
    https://www.kaggle.com/hrshtt/intro-to-indoor-location-navigation/
    who Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
    if not isinstance(txt_path, Path):
        path = Path(txt_path)
    
    _id, floor, filename = txt_path.parts[-3:]

    
    train_floor_data = read_data_file(txt_path)
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)

    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {_id}/{floor}/{filename}"
    )

def visualize_train_step_trajectory(txt_path, step_positions):
    """
    Edited from 
    https://www.kaggle.com/hrshtt/intro-to-indoor-location-navigation/
    who Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
#     _id, floor = path.split("/")[:2]
    if not isinstance(txt_path, Path):
        path = Path(txt_path)
    
    _id, floor, filename = txt_path.parts[-3:]

    
    train_floor_data = read_data_file(txt_path)
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)
        
    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {_id}/{floor}/{filename}", 
        trajectory2 = step_positions[:, 1:] #, step_positions[:, 2]
    )

In [ ]:
sample_txt_file_path = Path('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt')
sample_txt_file_path

**Predicts the position given the IMU and the starting point**

In [ ]:
def predict_position(trajectory_file): 
    # Copied and modified from https://github.com/location-competition/indoor-location-competition-20/blob/master/compute_f.py
    sample_data = read_data_file(str(trajectory_file))
    
    # GT position 
    waypoint_df = pd.DataFrame(sample_data.waypoint)
    waypoint_df.columns = ['timestamp', 'waypoint_x','waypoint_y']
    
    # Compute relative position
    step_timestamps, step_indexs, step_acce_max_mins = compute_steps(sample_data.acce)
    headings = compute_headings(sample_data.ahrs)
    stride_lengths = compute_stride_length(step_acce_max_mins)
    step_headings = compute_step_heading(step_timestamps, headings)
    rel_positions = compute_rel_positions(stride_lengths, step_headings)
    
    rel_positions[:, 1].cumsum(), rel_positions[:, 2].cumsum()
    
    predicted_positions = np.zeros(rel_positions.shape)
    predicted_positions[:, 0] = rel_positions[:, 0]
    predicted_positions[:, 1] = rel_positions[:, 1].cumsum() + waypoint_df.loc[0].waypoint_x
    predicted_positions[:, 2] = rel_positions[:, 2].cumsum() + waypoint_df.loc[0].waypoint_y
    
    return predicted_positions

def calc_errors(trajectory_file, predicted_positions=None, only_mean_error = True): 
    if predicted_positions is None: 
        predicted_positions = predict_position(trajectory_file)
       
    # GT position 
    sample_data = read_data_file(str(trajectory_file))
    
    errors = [] 
    time_diff = []
    start_time = sample_data.waypoint[0,0]
    for predicted_position in predicted_positions: 
        timediff = 10e15
        best_gt = -1
        for idx, gt in enumerate(sample_data.waypoint): 
            if abs(predicted_position[0] - gt[0]) < timediff: 
                timediff = abs(predicted_position[0] - gt[0])
                best_gt = gt
        errors.append(np.linalg.norm(predicted_position[1:] - best_gt[1:]))
        time_diff.append(predicted_position[0] - start_time)

    if only_mean_error: 
        return np.mean(errors)
    return errors, time_diff, np.mean(errors)

In [ ]:
predicted_positions = predict_position(sample_txt_file_path)

In [ ]:
calc_errors(sample_txt_file_path, predicted_positions)

In [ ]:
visualize_train_step_trajectory(Path(sample_txt_file_path), predicted_positions)

In [ ]:
import random
for filename in random.sample(os.listdir('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/'), 5): 
    sample_txt_file_path = os.path.join('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/', filename)
    predicted_positions = predict_position(sample_txt_file_path)
    print(calc_errors(sample_txt_file_path, predicted_positions))
    visualize_train_step_trajectory(Path(sample_txt_file_path), predicted_positions).show()
    plt.show()

In [ ]:
mean_errors = []
basedir = '../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/'
for filename in os.listdir(basedir):
    sample_txt_file_path = os.path.join(basedir, filename)
    predicted_positions = predict_position(sample_txt_file_path)
    mean_errors.append(calc_errors(sample_txt_file_path, predicted_positions))
np.mean(mean_errors), np.min(mean_errors), np.max(mean_errors), np.std(mean_errors)

**Calculate the error per floor and building**

In [ ]:
import collections
def calc_errors_per_building(building_path): 
    floor_mean_errors = collections.OrderedDict()
    for floor in os.listdir(building_path): 
        floor_mean_errors[floor] = []
        basedir = os.path.join(building_path, floor)
        for filename in os.listdir(basedir):
            sample_txt_file_path = os.path.join(basedir, filename)
            predicted_positions = predict_position(sample_txt_file_path)
            floor_mean_errors[floor] .append(calc_errors(sample_txt_file_path, predicted_positions))
    return floor_mean_errors

In [ ]:
floor_mean_errors = calc_errors_per_building('../input/indoor-location-navigation/train/5a0546857ecc773753327266/')

In [ ]:
labels, data = floor_mean_errors.keys(), floor_mean_errors.values()

plt.boxplot(data)
plt.xticks(range(1, len(labels) + 1), labels)
plt.show()

**Calculate the errors for 10 sample buildings**

In [ ]:
for building in random.sample(os.listdir('../input/indoor-location-navigation/train'), 10): 
    floor_mean_errors = calc_errors_per_building(os.path.join('../input/indoor-location-navigation/train', building))
    labels, data = floor_mean_errors.keys(), floor_mean_errors.values()
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.show()


# **waypoint path with gayro sensor & magnetic sensor**

In [ ]:
import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # ut(10 digit) -> date

# pandas settings -----------------------------------------
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# Graph drawing -------------------------------------------
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
from matplotlib_venn import venn2, venn2_circles
from matplotlib import animation as ani
from IPython.display import Image
from pylab import imread
from IPython.display import HTML

plt.rcParams["patch.force_edgecolor"] = True
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

%matplotlib inline
%config InlineBackend.figure_format='retina'

# ML -------------------------------------------
from sklearn.preprocessing import LabelEncoder

def unpickle(filename):
    with open(filename, 'rb') as fo:
        p = pickle.load(fo)
    return p

def to_pickle(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, -1)



class FeatureStore():
    
    # necessayr to re-check
    floor_convert = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}
    
    df_types = ['accelerometer',
                'gyroscope',
                'magnetic_field',
                'rotation_vector',
                'waypoint']
    
    # https://github.com/location-competition/indoor-location-competition-20
    df_type_cols = {'accelerometer': ["timestamp", "x", "y", "z", "accuracy"],
                'gyroscope': ["timestamp", "x", "y", "z", "accuracy"],
                'magnetic_field': ["timestamp", "x", "y", "z", "accuracy"],
                'rotation_vector': ["timestamp", "x", "y", "z", "accuracy"],
                'waypoint': ["timestamp", "x", "y"]}

    dtype_dict = {}
    dtype_dict["accelerometer"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                                   "accuracy":int}
    dtype_dict["gyroscope"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                               "accuracy":int}
    dtype_dict["magnetic_field"] = {"timestamp":int, "x":float, "y":float, 
                                    "z":float, "accuracy":int}
    dtype_dict["rotation_vector"] = {"timestamp":int, "x":float, "y":float, 
                                     "z":float, "accuracy":int}
    dtype_dict["waypoint"] = {"timestamp":int, "x":float, "y":float, "z":float}


    def __init__(self, site_id, floor, path_id, 
                 input_path="../input/indoor-location-navigation/",
                 save_path="../mid"):
        self.site_id = site_id.strip()
        self.floor = floor.strip()
        self.n_floor = self.floor_convert[self.floor]
        self.path_id = path_id.strip()
        
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
        self.save_path = save_path
        Path(save_path).mkdir(parents=True, exist_ok=True)
        
        self.site_info = SiteInfo(site_id=self.site_id, floor=self.floor, input_path=self.input_path)
        
    def _flatten(self, l):
        return list(itertools.chain.from_iterable(l))
    
    def multi_line_spliter(self, s):
        matches = re.finditer("TYPE_", s)
        matches_positions = [match.start() for match in matches]
        split_idx = [0] + [matches_positions[i]-14 for i in range(1, len(matches_positions))] + [len(s)]
        return [s[split_idx[i]:split_idx[i+1]] for i in range(len(split_idx)-1)]
    
    def load_df(self, ):
        path = str(Path(self.input_path)/f"train/{self.site_id}/{self.floor}/{self.path_id}.txt")
        with open(path) as f:
            data = f.readlines()
        
        modified_data = []
        for s in data:
            if s.count("TYPE_")>1:
                lines = self.multi_line_spliter(s)
                modified_data.extend(lines)
            else:
                modified_data.append(s)
        del data
        self.meta_info_len = len([d for d in modified_data if d[0]=="#"])
        self.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":") 
                                          for m in self._flatten([d.split("\t") 
                                                                  for d in modified_data if d[0]=="#"]) if m!="#"])

        data_df = pd.DataFrame([d.replace("\n", "").split("\t") for d in modified_data if d[0]!="#"])
        for dt in self.df_types:
            # select data type
            df_s = data_df[data_df[1]==f"TYPE_{dt.upper()}"]
            if len(df_s)==0:
                setattr(self, dt, pd.DataFrame(columns=self.df_type_cols[dt]))
            else:
                # remove empty cols
                na_info = df_s.isna().sum(axis=0) == len(df_s)
                df_s = df_s[[i for i in na_info[na_info==False].index if i!=1]].reset_index(drop=True)
                
                if len(df_s.columns)!=len(self.df_type_cols[dt]):
                    df_s.columns = self.df_type_cols[dt][:len(df_s.columns)]
                else:
                    df_s.columns = self.df_type_cols[dt]
            
                # set dtype          
                for c in df_s.columns:
                    df_s[c] = df_s[c].astype(self.dtype_dict[dt][c])
                                     
                # set DataFrame to attr
                setattr(self, dt, df_s)
    
    def get_site_info(self, keep_raw=False):
        self.site_info.get_site_info(keep_raw=keep_raw)
            
    def load_all_data(self, keep_raw=False):     
        self.load_df()
        self.get_site_info(keep_raw=keep_raw)
        
    def __getitem__(self, item):
        if item in self.df_types:
            return getattr(self, item)
        elif item=="sensors":
            try:
                return getattr(self, "sensor_df")
            except:
                self.sensor_df = pd.concat([feature["magnetic_field"].set_index("timestamp"), 
                                       feature["accelerometer"].set_index("timestamp"), 
                                       feature["gyroscope"].set_index("timestamp")], axis=1)
                if self.sensor_df.shape[1]==12:
                    self.sensor_df.columns = ["mag_x", "mag_y", "mag_z", "mag_acc", 
                                              "acc_x", "acc_y", "acc_z", "acc_acc",
                                              "gyr_x", "gyr_y", "gyr_z", "gyr_acc", ]
                else:
                    self.sensor_df.columns = ["mag_x", "mag_y", "mag_z", 
                                              "acc_x", "acc_y", "acc_z", 
                                              "gyr_x", "gyr_y", "gyr_z",  ]
                return self.sensor_df
        else:
            return None
    
    def save(self, ):
        # to be implemented
        pass
    
    
class SiteInfo():
    def __init__(self, site_id, floor, input_path="../input/indoor-location-navigation/"):
        self.site_id = site_id
        self.floor = floor
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
    def get_site_info(self, keep_raw=False):
        floor_info_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_info.json"
        with open(floor_info_path, "r") as f:
            self.floor_info = json.loads(f.read())
            self.site_height = self.floor_info["map_info"]["height"]
            self.site_width = self.floor_info["map_info"]["width"]
            if not keep_raw:
                del self.floor_info
            
        geojson_map_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/geojson_map.json"
        with open(geojson_map_path, "r") as f:
            self.geojson_map = json.loads(f.read())
            self.map_type = self.geojson_map["type"]
            self.features = self.geojson_map["features"]
            
            self.floor_coordinates = self.features[0]["geometry"]["coordinates"]
            self.store_coordinates = [self.features[i]["geometry"]["coordinates"] 
                                          for i in range(1, len(self.features))]
                
            if not keep_raw:
                del self.geojson_map
    
    def show_site_image(self, ax):
        path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_image.png"
        ax.imshow(imread(path), extent=[0, self.site_width, 0, self.site_height])

    def draw_polygon(self, size=8, only_floor=False):

        fig = plt.figure()
        ax = plt.subplot(111)
            
        xmax, xmin, ymax, ymin = self._draw(self.floor_coordinates, ax, calc_minmax=True)
        if not only_floor:
            self._draw(self.store_coordinates, ax, fill=True)
        plt.legend([])
        
        xrange = xmax - xmin
        yrange = ymax - ymin
        ratio = yrange / xrange
        
        self.x_size = size
        self.y_size = size*ratio

        fig.set_figwidth(size)
        fig.set_figheight(size*ratio)
        # plt.show()
        return ax
        
    def _draw(self, coordinates, ax, fill=False, calc_minmax=False):
        xmax, ymax = -np.inf, -np.inf
        xmin, ymin = np.inf, np.inf
        for i in range(len(coordinates)):
            ndim = np.ndim(coordinates[i])
            if ndim==2:
                corrd_df = pd.DataFrame(coordinates[i])
                if fill:
                    ax.fill(corrd_df[0], corrd_df[1], alpha=0.7)
                else:
                    corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                if calc_minmax:
                    xmax = max(xmax, corrd_df[0].max())
                    xmin = min(xmin, corrd_df[0].min())

                    ymax = max(ymax, corrd_df[1].max())
                    ymin = min(ymin, corrd_df[1].min())
            elif ndim==3:
                for j in range(len(coordinates[i])):
                    corrd_df = pd.DataFrame(coordinates[i][j])
                    if fill:
                        ax.fill(corrd_df[0], corrd_df[1], alpha=0.6)
                    else:
                        corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                    if calc_minmax:
                        xmax = max(xmax, corrd_df[0].max())
                        xmin = min(xmin, corrd_df[0].min())

                        ymax = max(ymax, corrd_df[1].max())
                        ymin = min(ymin, corrd_df[1].min())
            else:
                assert False, f"ndim of coordinates should be 2 or 3: {ndim}"
        if calc_minmax:
            return xmax, xmin, ymax, ymin
        else:
            return None
        

In [ ]:
from matplotlib import animation as ani
import functools
from IPython.display import Image
plt.rcParams["font.size"] = 7

def animate(axes, nframe):
    global num_frame
    
    ts = feature.waypoint.timestamp.values[nframe]
    
    # axes[0]------------------------------
    wp =feature.waypoint.iloc[nframe]
    axes[0].text(x=wp.x, y=wp.y, s=f"{nframe}", fontsize=6)
    
    # axes[1]------------------------------
    axes[1].vlines(ts, -3.5, 3.5, "k", lw=1)
    axes[1].text(ts, -1, f"{nframe}", fontsize=8)
    
    # axes[2]------------------------------
    axes[2].vlines(ts, -40, 40, "k", lw=1)
    axes[2].text(ts, -20, f"{nframe}", fontsize=8)


def drow_path_gyr_mag(feature, gif_name):    
    ts_min = feature.waypoint.timestamp.min()
    ts_max = feature.waypoint.timestamp.max()
    query = f"{ts_min} <= timestamp and timestamp <= {ts_max}"
    sensor_df = feature["sensors"].reset_index().query(query)
    sensor_df["ts_diff"] = sensor_df.timestamp.diff() / 1000
    sensor_df["x_pos"] = 0
    sensor_df["y_pos"] = 0

    sensor_df["gyr_z_dif_ts"] = sensor_df["gyr_z"] * sensor_df["ts_diff"].fillna(0.02)
    fig, axes = plt.subplots(3, 1,
        gridspec_kw=dict(width_ratios=[1], 
                         height_ratios=[4,1,1], 
                         wspace=0.1, 
                         hspace=0.3),)
    # axes[0]------------------------------
    feature.site_info.show_site_image(ax=axes[0])
    axes[0].grid(False)
    axes[0].tick_params(labelsize=7)
    # axes[1]------------------------------
    gyro_data = sensor_df.set_index("timestamp").gyr_z_dif_ts.cumsum()
    gyro_data.plot(lw=1, label="gyr_z_dif_ts", ax=axes[1])
    axes[1].hlines(np.pi, ts_min, ts_max, "gray", linestyle="--", lw=0.5)
    axes[1].hlines(-np.pi, ts_min, ts_max, "gray", linestyle="--", lw=0.5)
    axes[1].legend(loc="best", fontsize=8)
    axes[1].set_title("gyr_z", fontsize=8)
    axes[1].set_xlabel('timestamp', fontsize=8)
    axes[1].tick_params(labelsize=7)
    # axes[2]------------------------------
    sensor_df.set_index("timestamp").mag_x.plot(lw=1, label="mag_x", ax=axes[2])
    sensor_df.set_index("timestamp").mag_y.plot(lw=1, label="mag_y", ax=axes[2])
    axes[2].legend(loc="best", fontsize=8)
    axes[2].set_title("magnetic_field_x, y", fontsize=8)
    axes[2].set_xlabel('timestamp', fontsize=8)
    axes[2].tick_params(labelsize=7)
    
    animate_ = functools.partial(animate, axes)
    anim = ani.FuncAnimation(fig, animate_, frames=len(feature.waypoint))

    fig.set_figwidth(8)
    fig.set_figheight(12)
    anim.save(gif_name, fps=2, dpi=256)
    plt.close()

In [ ]:
feature = FeatureStore(site_id='5da138764db8ce0c98bcaa46', floor='F1', path_id="5daae24718410e00067e6d44")
feature.load_all_data()

In [ ]:
drow_path_gyr_mag(feature, 'path_anim01.gif')

In [ ]:
Image('path_anim01.gif')

In [ ]:
feature = FeatureStore(site_id='5cd56b5ae2acfd2d33b5854a', floor='F1', path_id="5d076ecf0e86b60008036124")
feature.load_all_data()
drow_path_gyr_mag(feature, 'path_anim03.gif')

In [ ]:
Image('path_anim03.gif')